In [1]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipywidgets import interact
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

def random_color():
    color = np.random.randint(16, 256, size=3)
    color = [str(hex(i))[2:] for i in color]
    return '#' + ''.join(color).upper()

def create_point(row):
    return Point(row['LATITUDE'], row['LONGITUDE'])

In [2]:
data = pd.read_csv("bike_share_data/cleaned_data.csv")
data['point'] = data.apply(create_point, axis=1)

### copy of class code only for debuging purposes

In [3]:
import pandas as pd
from shapely import intersection
from shapely.geometry import MultiPoint, Polygon, Point
from shapely import voronoi_polygons

class weightedVoroniDigram:
    """
    Class that computes weighted voronoi digrams for our use case
    
    """
    def __init__(self, df):
        """
        """
        self.data = df

    def find_optimal_voronoi_digram(num_to_add, num_to_remove, df, boundary):
        """
        Args:
            num_to_add (int): the number of points to add to data
            num_to_remove (int): number of points in dataset to remove
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.
        """
        
        removed_df = weightedVoroniDigram.__find_optimal_point_to_remove(num_to_remove, df, boundary)
        added_removed_df = weightedVoroniDigram.__find_optimal_points_to_add(num_to_add, removed_df, boundary)

        
        
            
        return weightedVoroniDigram.__weighted_voronoi_polygons(added_removed_df, boundary)
        

    def __find_optimal_point_to_remove(num_to_remove, df, boundary):
        """
        Args:
            num_to_remove (int): number of points in dataset to remove
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.

        return, dataframe of with the num_to_remove points removed that were the worst locations
        """
        return df

    def __find_optimal_points_to_add(num_to_add, df, boundary):
        """
        Probably use regression to compute the anticipated weights for the new points to be added.

        
        Args:
            num_to_add (int): the number of points to add to data
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y).
            boundary (shapley polygon): Bound the Vorinoi digram to this region.

        return, dataframe of with the num_to_add points added that were the best locations
        """
        return df
        
    def __weighted_voronoi_polygons(df, boundary):
        """
        Given a dataframe with weights compute the shapley poloygons for the weighted voronoi digram on the pts.
        
        Args:
            df (dataframe): dataframe with a column for "weights", "station_name", and the "point" (x,y)
            boundary (shapley polygon): Bound the Vorinoi digram to this region.
        Returns:
            dataframe: with the columns "poly", column of Shapley Poloygons that is the voronoi digram for the point, 
                                                    "weights", "station_name", and the "point" (x,y)
        """
        points2 =  MultiPoint(df['point'].to_list())
       
        voronoi_diagram = voronoi_polygons(points2)

        vor_polygons = []
        for geom in voronoi_diagram.geoms:
            xx, yy = geom.exterior.coords.xy
            coord = list(zip(xx, yy))
            inter = intersection(geom, boundary)
            xx, yy = inter.exterior.coords.xy
            coord_clipped = list(zip(xx, yy))
            
            vor_polygons.append(coord_clipped)

        df['poly'] = vor_polygons
        return df



### Voronoi digram plotter

In [4]:
# from voronoi import weightedVoroniDigram

In [5]:
def generate_points(n):
    return data[['LATITUDE','LONGITUDE']].to_numpy()
    
def plot_voronoi(df):
    # Create Folium map
    points = df[['LATITUDE','LONGITUDE']].to_numpy()
    map_center = points.mean(axis=0)
    m = folium.Map(location=map_center, zoom_start=10)

    for index, row in df.iterrows():
        folium.Polygon(
                    locations=row['poly'],
                    color="blue",
                    weight=4,
                    fill_color=random_color(),
                    fill_opacity=0.5,
                    fill=True,
                    popup=f"{row['station_name']}",
                    ).add_to(m)
        
        folium.CircleMarker(location=[row['LATITUDE'],row['LONGITUDE']],
                        radius=2,
                        weight=2).add_to(m)
    
    # Display map
    display(m)

@interact(Add=(0, 10), Remove=(0,10))
def adjust_points(Add, Remove):
    boundary = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
    points = weightedVoroniDigram.find_optimal_voronoi_digram(Add, Remove, data, boundary)
    # points = generate_points(Add)
    plot_voronoi(points)


interactive(children=(IntSlider(value=5, description='Add', max=10), IntSlider(value=5, description='Remove', …

In [6]:
boundary = Polygon([(38.7, -76.8), (38.7,-77.4), (39.2,-77.4), (39.2,-76.8)])
points = weightedVoroniDigram.find_optimal_voronoi_digram(0, 0, data, boundary)

In [20]:
poly_w = []
points['weight'] = points['count_rides_ending_at']+points['count_rides_starting_at']

for index, row in points.iterrows():
    poly_w.append(Polygon(row['poly']).area*row['weight'])


points['poly_weight'] = poly_w
points.sort_values(by='poly_weight', ascending=False)
# points[points['poly_weight']==max(points['poly_weight'])]

,Unnamed: 0,STATION_ID,NUM_DOCKS_AVAILABLE,NUM_DOCKS_DISABLED,NUM_BIKES_AVAILABLE,NUM_EBIKES_AVAILABLE,NUM_BIKES_DISABLED,IS_RENTING,HAS_KIOSK,IOS,...,count_rides_starting_at,count_of_member_rides_starting_at,ending_at_avg_ride_duration,count_rides_ending_at,count_of_member_rides_ending_at,station_name,point,poly,weight,poly_weight
739,739,082522f0-1f3f-11e7-bf6b-3863bb334450,23,0,11,0,2,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,25224.0,6039.0,0 days 00:39:57.707262,26218.0,6766.0,Smithsonian-National Mall / Jefferson Dr & 12t...,POINT (38.888774 -77.028694),"[(39.04500157083292, -77.05148797733611), (39....",51442.0,262.969868
76,76,08254284-1f3f-11e7-bf6b-3863bb334450,22,0,1,0,2,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,32559.0,5753.0,0 days 00:34:16.167755,32452.0,6514.0,Lincoln Memorial,POINT (38.888255 -77.049437),"[(38.864260968852385, -77.37786067076), (38.92...",65011.0,178.706070
741,741,0824749f-1f3f-11e7-bf6b-3863bb334450,13,0,17,0,1,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,33344.0,23546.0,0 days 00:11:30.705995,31792.0,21992.0,14th & V St NW,POINT (38.917765 -77.032096),"[(39.14871119174583, -77.05889396467398), (39....",65136.0,165.998263
671,671,08250172-1f3f-11e7-bf6b-3863bb334450,20,0,5,0,1,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,10664.0,7536.0,0 days 00:18:21.479375,11079.0,7943.0,18th St & Pennsylvania Ave NW,POINT (38.89968 -77.041539),"[(39.0307233216647, -77.1878780716923), (39.04...",21743.0,161.009500
688,688,7995da4d-4d99-4797-bb9f-37a51a28cc09,17,0,0,0,2,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,5748.0,4195.0,0 days 00:13:14.237506,5823.0,4306.0,North Capitol & K St NE,POINT (38.902444 -77.008246),"[(38.97897163483206, -77.33040490789101), (39....",11571.0,146.485354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,412,1929488700232798842,11,0,1,0,0,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (38.8348 -77.307057),"[(38.90181802991936, -77.00357544187545), (38....",NaN,NaN
430,430,1929517249726509974,9,0,3,3,0,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (38.831211 -77.308069),"[(38.902954718548195, -77.03724090660654), (38...",NaN,NaN
506,506,1926924117062925284,13,0,6,1,0,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (38.912181 -77.066074),"[(38.909282801273896, -76.95962171525952), (38...",NaN,NaN
702,702,1929488402496518868,10,0,2,0,0,YES,YES,https://dc.lft.to/lastmile_qr_scan,...,NaN,NaN,NaN,NaN,NaN,NaN,POINT (38.827501 -77.304157),"[(38.98502822224287, -77.09862837622272), (38....",NaN,NaN
